# Homework №1

Load reads from all experiments from article **[Evaluation of two main RNA-seq approaches for gene quantification in clinical RNA sequencing: polyA+ selection versus rRNA depletion](https://www.nature.com/articles/s41598-018-23226-4)**. 

In this article authors compared two methods of removing highly abundant ribosomal RNAs (rRNA): positive polyA+ selection and rRNA depletion (negative selection). They evaluated these two RNA sequencing approaches using human blood and colon tissue samples.

Use the Nextflow pipeline.

## Let's start!
1. Where can I find data that they used in this study?

> All the raw sequencing reads generated in this study have been submitted to the NCBI Sequence Read Archive and are available under the accession number **SRP127360**.
>

2. Download file with a list of accessions: 

Go to NCBI → Choose “SRA” as Database and look for “SRP127360” → Click “Send to” → Choose Destination - “File” → Format - “Accession List”
After that you will obtain file called SraAccList.txt with 16 accession numbers.

3. Install needed tools and programs.

In [ ]:
# Install SRA toolkit 
! curl --output sratoolkit.tar.gz https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/current/sratoolkit.current-mac64.tar.gz
! tar -vxzf sratoolkit.tar.gz
! vdb-config -i 

# Install FastQC
! curl --output fastqc_v0.11.9.zip https://www.bioinformatics.babraham.ac.uk/projects/fastqc/fastqc_v0.11.9.zip
! unzip fastqc_v0.11.9.zip
! chmod +x FastQC/fastqc

# Install MultyQC
! pip3 install multiqc

# Install NextFlow
! curl -fsSL https://get.nextflow.io | bash
    
# For opening .dot files
brew install graphviz 

## Versions of programs and OS version of local machine
* SRA toolkit: sratoolkit.3.0.0-mac64
* FastQC: fastqc_v0.11.9 
* MultiQC: version 1.13 (6e2b162)
* Nextflow: version 22.04.5 build 5708
* MacOS Version:	12.3.1
* MacOS BuildVersion:	21E258

In [1]:
# Add path to programs in ${PATH}
! export PATH=$PATH:/Users/mar4/transcriptomics/software/sratoolkit.3.0.0-mac64/bin
! export PATH=$PATH:/Users/mar4/transcriptomics/software/FastQC
! export PATH=$PATH:/Users/mar4/transcriptomics/software/

## Test run for two accessions from lecture
Every downloaded tool has run succesfully. 

In [3]:
# Download two files 
! fasterq-dump --split-files SRR20851170
! fasterq-dump --split-files SRR20851171

# Compress four files
! bgzip SRR20851170_1.fastq
! bgzip SRR20851170_2.fastq
! bgzip SRR20851171_1.fastq
! bgzip SRR20851171_2.fastq

# Run FastQC (We can use all files as input)
! fastqc SRR20851170_1.fastq.gz SRR20851170_2.fastq.gz SRR20851171_1.fastq.gz SRR20851171_2.fastq.gz

# Run MultiQC (We should give directory with files after running FastQC)
! multiqc .

spots read      : 135,014
reads read      : 270,028
reads written   : 270,028
spots read      : 158,874
reads read      : 317,748
reads written   : 317,748
Started analysis of SRR20851170_1.fastq.gz
Approx 5% complete for SRR20851170_1.fastq.gz
Approx 10% complete for SRR20851170_1.fastq.gz
Approx 15% complete for SRR20851170_1.fastq.gz
Approx 20% complete for SRR20851170_1.fastq.gz
Approx 25% complete for SRR20851170_1.fastq.gz
Approx 30% complete for SRR20851170_1.fastq.gz
Approx 35% complete for SRR20851170_1.fastq.gz
Approx 40% complete for SRR20851170_1.fastq.gz
Approx 45% complete for SRR20851170_1.fastq.gz
Approx 50% complete for SRR20851170_1.fastq.gz
Approx 55% complete for SRR20851170_1.fastq.gz
Approx 60% complete for SRR20851170_1.fastq.gz
Approx 65% complete for SRR20851170_1.fastq.gz
Approx 70% complete for SRR20851170_1.fastq.gz
Approx 75% complete for SRR20851170_1.fastq.gz
Approx 80% complete for SRR20851170_1.fastq.gz
Approx 85% complete for SRR20851170_1.fastq.gz
App

## Run Nextflow script and check the results
There are 16 accessions. It is too much for my computer, so I decided to run pipeline on two accession: one sample is from colon (SRR6410610) and the other is from blood (SRR6410618).

As input nextflow pipeline use file SraAccList.txt where every line is separated accession number.
Nextflow script download FastQ files with fasterq-dump, then evaluate quality with FastQC and join its results with MultiQC. Output is MultiQC html report.

**Flags:**<br />
-with-report - create processes execution html report.<br />
-with-dag - create pipeline DAG file.

In [4]:
! nextflow run hw1_intro.nf -with-report results/nextflow_report.html -with-dag results/flowchart.pdf --SRA_acc_list 'SraAccList.txt'

N E X T F L O W  ~  version 22.04.5
Launching `hw1_intro.nf` [infallible_mendel] DSL2 - revision: 2eb2a7558d

  Q U A L I T Y   C O N T R O L  
SRA numbers in file        : SraAccList.txt
Results location   : results/multiqc_report/
executor >  local (2)
[32/38ed37] process > DownloadFastQ (1) [  0%] 0 of 2
[-        ] process > RunFastQC         -
[-        ] process > RunMultiQC        -
WARN: Task runtime metrics are not reported when using macOS without a container engine

executor >  local (2)
[32/38ed37] process > DownloadFastQ (1) [  0%] 0 of 2
[-        ] process > RunFastQC         -
[-        ] process > RunMultiQC        -
WARN: Task runtime metrics are not reported when using macOS without a container engine

executor >  local (3)
[89/496d2f] process > DownloadFastQ (2) [ 50%] 1 of 2
[d1/ea5379] process > RunFastQC (1)     [  0%] 0 of 1
[-        ] process > RunMultiQC        -
WARN: Task runtime metrics are not reported when using macOS without a container engine


executo

Also we can look at hw1_intro.nf!

In [8]:
! cat hw1_intro.nf

#!/usr/bin/env nextflow

params.SRA_acc_list = "filename.txt"
params.results_dir = "results/multiqc_report/"

log.info ""
log.info "  Q U A L I T Y   C O N T R O L  "
log.info "================================="
log.info "SRA numbers in file        : ${params.SRA_acc_list}"
log.info "Results location   : ${params.results_dir}"

File acc_list = new File(params.SRA_acc_list)
SRA_list = acc_list.getText('UTF-8').split("\n")

process DownloadFastQ {
  input:
    val sra_acc

  output:
    path "${sra_acc}/*"

  script:
    """
    fasterq-dump --split-files -O ${sra_acc}/ ${sra_acc} 
    gzip -r ${sra_acc}/
    """
}

process RunFastQC {
  input:
    path fastq_files

  output:
    path "qc/*"

  script:
    """
    mkdir qc
    fastqc -o qc $fastq_files
    """
}

process RunMultiQC {
publishDir "${params.results_dir}"
  input:
    path fastqc_out
  output:
    path "multiqc_report.html"
  script:
    """
    multiqc ${fastqc_out}
    """
}

workflow {
  data = Channel.of( SRA_list )
  Do